In [1]:
%matplotlib inline

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Introduction
------------
steps:

#. Load the IMDB dataset
#. Train a variety of Doc2Vec models on the dataset
#. Evaluate the performance of each model using a logistic regression
#. Examine some of the results directly:

When examining results, we will look for answers for the following questions:

#. Are inferred vectors close to the precalculated ones?
#. Do close documents seem more related than distant ones?
#. Do the word vectors show useful similarities?
#. Are the word vectors from this dataset any good at analogies?

Load corpus
-----------

data is available on `IMDB archive
<http://ai.stanford.edu/~amaas/data/sentiment/>`_.

Each review is a single line of text containing multiple sentences, for example:

```
One of the best movie-dramas I have ever seen. We do a lot of acting in the
church and this is one that can be used as a resource that highlights all the
good things that actors can do in their work. I highly recommend this one,
especially for those who have an interest in acting, as a "must see."
```

These reviews will be the **documents** that we will work with in this tutorial.
There are 100 thousand reviews in total.

#. 25k reviews for training (12.5k positive, 12.5k negative)
#. 25k reviews for testing (12.5k positive, 12.5k negative)
#. 50k unlabeled reviews

Out of 100k reviews, 50k have a label: either positive (the reviewer liked
the movie) or negative.
The remaining 50k are unlabeled.


#. Download the tar.gz file (it's only 84MB, so this shouldn't take too long)
#. Unpack it and extract each movie review
#. Split the reviews into training and test datasets

define a convenient datatype for holding data for a single document:

* words: The text of the document, as a ``list`` of words.
* tags: Used to keep the index of the document in the entire dataset.
* split: one of ``train``\ , ``test`` or ``extra``. Determines how the document will be used (for training, testing, etc).
* sentiment: either 1 (positive), 0 (negative) or None (unlabeled document).

This data type is helpful for later evaluation and reporting.
In particular, the ``index`` member will help us quickly and easily retrieve the vectors for a document from a model.




In [3]:
import collections

SentimentDocument = collections.namedtuple('SentimentDocument', 'words tags split sentiment')

In [7]:
! pip install gensim

We can now proceed with loading the corpus.



In [8]:
import io
import re
import tarfile
import os.path

import smart_open
import gensim.utils

def download_dataset(url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'):
    fname = url.split('/')[-1]

    if os.path.isfile(fname):
       return fname

    # Download the file to local storage first.
    # We can't read it on the fly because of
    # https://github.com/RaRe-Technologies/smart_open/issues/331
    with smart_open.open(url, "rb", ignore_ext=True) as fin:
        with smart_open.open(fname, 'wb', ignore_ext=True) as fout:
            while True:
                buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                if not buf:
                    break
                fout.write(buf)

    return fname

def create_sentiment_document(name, text, index):
    _, split, sentiment_str, _ = name.split('/')
    sentiment = {'pos': 1.0, 'neg': 0.0, 'unsup': None}[sentiment_str]

    if sentiment is None:
        split = 'extra'

    tokens = gensim.utils.to_unicode(text).split()
    return SentimentDocument(tokens, [index], split, sentiment)

def extract_documents():
    fname = download_dataset()

    index = 0

    with tarfile.open(fname, mode='r:gz') as tar:
        for member in tar.getmembers():
            if re.match(r'aclImdb/(train|test)/(pos|neg|unsup)/\d+_\d+.txt$', member.name):
                member_bytes = tar.extractfile(member).read()
                member_text = member_bytes.decode('utf-8', errors='replace')
                assert member_text.count('\n') == 0
                yield create_sentiment_document(member.name, member_text, index)
                index += 1

alldocs = list(extract_documents())

Here's what a single document looks like



In [9]:
print(alldocs[27])

SentimentDocument(words=['I', 'was', 'looking', 'forward', 'to', 'this', 'movie.', 'Trustworthy', 'actors,', 'interesting', 'plot.', 'Great', 'atmosphere', 'then', '?????', 'IF', 'you', 'are', 'going', 'to', 'attempt', 'something', 'that', 'is', 'meant', 'to', 'encapsulate', 'the', 'meaning', 'of', 'life.', 'First.', 'Know', 'it.', 'OK', 'I', 'did', 'not', 'expect', 'the', 'directors', 'or', 'writers', 'to', 'actually', 'know', 'the', 'meaning', 'but', 'I', 'thought', 'they', 'may', 'have', 'offered', 'crumbs', 'to', 'peck', 'at', 'and', 'treats', 'to', 'add', 'fuel', 'to', 'the', 'fire-Which!', 'they', 'almost', 'did.', 'Things', 'I', "didn't", 'get.', 'A', 'woman', 'wandering', 'around', 'in', 'dark', 'places', 'and', 'lonely', 'car', 'parks', 'alone-oblivious', 'to', 'the', 'consequences.', 'Great', 'riddles', 'that', 'fell', 'by', 'the', 'wayside.', 'The', 'promise', 'of', 'the', 'knowledge', 'therein', 'contained', 'by', 'the', 'original', 'so-called', 'criminal.', 'I', 'had', 'no

Extract our documents and split into training/test sets



In [10]:
train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
print('%d docs: %d train-sentiment, %d test-sentiment' % (len(alldocs), len(train_docs), len(test_docs)))

100000 docs: 25000 train-sentiment, 25000 test-sentiment


Set-up Doc2Vec Training & Evaluation Models
-------------------------------------------


We vary the following parameter choices:

* 100-dimensional vectors, as the 400-d vectors of the paper take a lot of
  memory and, in our tests of this task, don't seem to offer much benefit
* Similarly, frequent word subsampling seems to decrease sentiment-prediction
  accuracy, so it's left out
* ``cbow=0`` means skip-gram which is equivalent to the paper's 'PV-DBOW'
  mode, matched in gensim with ``dm=0``
* Added to that DBOW model are two DM models, one which averages context
  vectors (\ ``dm_mean``\ ) and one which concatenates them (\ ``dm_concat``\ ,
  resulting in a much larger, slower, more data-hungry model)
* A ``min_count=2`` saves quite a bit of model memory, discarding only words
  that appear in a single doc (and are thus no more expressive than the
  unique-to-each doc vectors themselves)




In [11]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

common_kwargs = dict(
    vector_size=100, epochs=20, min_count=2,
    sample=0, workers=multiprocessing.cpu_count(), negative=5, hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

2019-11-21 17:03:27,612 : INFO : using concatenative 1100-dimensional layer1
2019-11-21 17:03:27,614 : INFO : collecting all words and their counts
2019-11-21 17:03:27,615 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-11-21 17:03:28,227 : INFO : PROGRESS: at example #10000, processed 2292381 words (3747805/s), 150816 word types, 10000 tags
2019-11-21 17:03:28,804 : INFO : PROGRESS: at example #20000, processed 4573645 words (3955736/s), 238497 word types, 20000 tags
2019-11-21 17:03:29,424 : INFO : PROGRESS: at example #30000, processed 6865575 words (3706859/s), 312348 word types, 30000 tags
2019-11-21 17:03:30,056 : INFO : PROGRESS: at example #40000, processed 9190019 words (3683341/s), 377231 word types, 40000 tags
2019-11-21 17:03:30,713 : INFO : PROGRESS: at example #50000, processed 11557847 words (3606381/s), 438729 word types, 50000 tags
2019-11-21 17:03:31,335 : INFO : PROGRESS: at example #60000, processed 13899883 words (3772862/s), 49

Doc2Vec(dbow,d100,n5,mc2,t4) vocabulary scanned & state initialized


2019-11-21 17:03:42,664 : INFO : PROGRESS: at example #10000, processed 2292381 words (3938447/s), 150816 word types, 10000 tags
2019-11-21 17:03:43,176 : INFO : PROGRESS: at example #20000, processed 4573645 words (4468265/s), 238497 word types, 20000 tags
2019-11-21 17:03:43,754 : INFO : PROGRESS: at example #30000, processed 6865575 words (3965285/s), 312348 word types, 30000 tags
2019-11-21 17:03:44,336 : INFO : PROGRESS: at example #40000, processed 9190019 words (4007503/s), 377231 word types, 40000 tags
2019-11-21 17:03:44,922 : INFO : PROGRESS: at example #50000, processed 11557847 words (4038083/s), 438729 word types, 50000 tags
2019-11-21 17:03:45,478 : INFO : PROGRESS: at example #60000, processed 13899883 words (4235782/s), 493913 word types, 60000 tags
2019-11-21 17:03:46,086 : INFO : PROGRESS: at example #70000, processed 16270094 words (3898611/s), 548474 word types, 70000 tags
2019-11-21 17:03:46,650 : INFO : PROGRESS: at example #80000, processed 18598876 words (414105

Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4) vocabulary scanned & state initialized


2019-11-21 17:03:57,185 : INFO : PROGRESS: at example #10000, processed 2292381 words (4246788/s), 150816 word types, 10000 tags
2019-11-21 17:03:57,732 : INFO : PROGRESS: at example #20000, processed 4573645 words (4180321/s), 238497 word types, 20000 tags
2019-11-21 17:03:58,263 : INFO : PROGRESS: at example #30000, processed 6865575 words (4331546/s), 312348 word types, 30000 tags
2019-11-21 17:03:58,850 : INFO : PROGRESS: at example #40000, processed 9190019 words (3964787/s), 377231 word types, 40000 tags
2019-11-21 17:03:59,427 : INFO : PROGRESS: at example #50000, processed 11557847 words (4113837/s), 438729 word types, 50000 tags
2019-11-21 17:03:59,972 : INFO : PROGRESS: at example #60000, processed 13899883 words (4311051/s), 493913 word types, 60000 tags
2019-11-21 17:04:00,588 : INFO : PROGRESS: at example #70000, processed 16270094 words (3856949/s), 548474 word types, 70000 tags
2019-11-21 17:04:01,136 : INFO : PROGRESS: at example #80000, processed 18598876 words (426227

Doc2Vec(dm/c,d100,n5,w5,mc2,t4) vocabulary scanned & state initialized


Le and Mikolov note that combining a paragraph vector from Distributed Bag of
Words (DBOW) and Distributed Memory (DM) improves performance. We will
follow, pairing the models together for evaluation. Here, we concatenate the
paragraph vectors obtained from each model with the help of a thin wrapper
class included in a gensim test module.



In [15]:
!pip install testfixtures

In [16]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

2019-11-21 17:06:36,529 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-21 17:06:36,534 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


Predictive Evaluation Methods
-----------------------------

Given a document, our ``Doc2Vec`` models output a vector representation of the document.
How useful is a particular model?
In case of sentiment analysis, we want the ouput vector to reflect the sentiment in the input document.
So, in vector space, positive documents should be distant from negative documents.

We train a logistic regression from the training set:

  - regressors (inputs): document vectors from the Doc2Vec model
  - target (outpus): sentiment labels

So, this logistic regression will be able to predict sentiment given a document vector.

Next, we test our logistic regression on the test set, and measure the rate of errors (incorrect predictions).
If the document vectors from the Doc2Vec model reflect the actual sentiment well, the error rate will be low.

Therefore, the error rate of the logistic regression is indication of *how well* the given Doc2Vec model represents documents as vectors.
We can then compare different ``Doc2Vec`` models by looking at their error rates.




In [17]:
import numpy as np
import statsmodels.api as sm
from random import sample

def logistic_predictor_from_data(train_targets, train_regressors):
    """Fit a statsmodel logistic predictor on supplied data"""
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets = [doc.sentiment for doc in train_set]
    train_regressors = [test_model.docvecs[doc.tags[0]] for doc in train_set]
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_set]
    test_regressors = sm.add_constant(test_regressors)

    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_set])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

Bulk Training & Per-Model Evaluation
------------------------------------

Note that doc-vector training is occurring on *all* documents of the dataset,
which includes all TRAIN/TEST/DEV docs.  Because the native document-order
has similar-sentiment documents in large clumps – which is suboptimal for
training – we work with once-shuffled copy of the training set.

We evaluate each model's sentiment predictive power based on error rate, and
the evaluation is done for each model.

In [18]:
from collections import defaultdict
error_rates = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [19]:
from random import shuffle
shuffled_alldocs = alldocs[:]
shuffle(shuffled_alldocs)

for model in simple_models:
    print("Training %s" % model)
    model.train(shuffled_alldocs, total_examples=len(shuffled_alldocs), epochs=model.epochs)

    print("\nEvaluating %s" % model)
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print("\n%f %s\n" % (err_rate, model))

for model in [models_by_name['dbow+dmm'], models_by_name['dbow+dmc']]:
    print("\nEvaluating %s" % model)
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print("\n%f %s\n" % (err_rate, model))

2019-11-21 17:07:00,563 : INFO : training model with 4 workers on 265408 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=5 window=5


Training Doc2Vec(dbow,d100,n5,mc2,t4)


2019-11-21 17:07:01,601 : INFO : EPOCH 1 - PROGRESS: at 3.88% examples, 900425 words/s, in_qsize 8, out_qsize 0
2019-11-21 17:07:02,609 : INFO : EPOCH 1 - PROGRESS: at 8.43% examples, 977406 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:07:03,618 : INFO : EPOCH 1 - PROGRESS: at 13.14% examples, 1007538 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:07:04,618 : INFO : EPOCH 1 - PROGRESS: at 17.90% examples, 1026540 words/s, in_qsize 8, out_qsize 0
2019-11-21 17:07:05,619 : INFO : EPOCH 1 - PROGRESS: at 22.69% examples, 1037116 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:07:06,620 : INFO : EPOCH 1 - PROGRESS: at 27.28% examples, 1040022 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:07:07,623 : INFO : EPOCH 1 - PROGRESS: at 32.04% examples, 1044958 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:07:08,623 : INFO : EPOCH 1 - PROGRESS: at 36.58% examples, 1048531 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:07:09,626 : INFO : EPOCH 1 - PROGRESS: at 40.78% examples, 1038293 words/s, in

2019-11-21 17:08:06,836 : INFO : EPOCH 3 - PROGRESS: at 86.74% examples, 896661 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:08:07,849 : INFO : EPOCH 3 - PROGRESS: at 91.04% examples, 899556 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:08:08,852 : INFO : EPOCH 3 - PROGRESS: at 95.22% examples, 901582 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:08:09,881 : INFO : EPOCH 3 - PROGRESS: at 98.99% examples, 899400 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:08:10,129 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-21 17:08:10,138 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-21 17:08:10,142 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-21 17:08:10,147 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-21 17:08:10,150 : INFO : EPOCH - 3 : training on 23279529 raw words (22951015 effective words) took 25.5s, 898960 effective words/s
2019-11-21 17:08:11,178 : INFO : EPOCH 4 - P

2019-11-21 17:09:08,718 : INFO : EPOCH 6 - PROGRESS: at 27.16% examples, 1029034 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:09,722 : INFO : EPOCH 6 - PROGRESS: at 31.22% examples, 1013373 words/s, in_qsize 8, out_qsize 0
2019-11-21 17:09:10,743 : INFO : EPOCH 6 - PROGRESS: at 34.96% examples, 992157 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:11,744 : INFO : EPOCH 6 - PROGRESS: at 39.10% examples, 990263 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:12,745 : INFO : EPOCH 6 - PROGRESS: at 43.67% examples, 994669 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:13,748 : INFO : EPOCH 6 - PROGRESS: at 48.15% examples, 998295 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:14,750 : INFO : EPOCH 6 - PROGRESS: at 52.69% examples, 1002068 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:15,751 : INFO : EPOCH 6 - PROGRESS: at 57.26% examples, 1005612 words/s, in_qsize 7, out_qsize 0
2019-11-21 17:09:16,753 : INFO : EPOCH 6 - PROGRESS: at 61.73% examples, 1007706 words/s, in

...

2019-11-21 18:08:44,238 : INFO : EPOCH 17 - PROGRESS: at 16.04% examples, 404626 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:08:45,252 : INFO : EPOCH 17 - PROGRESS: at 17.90% examples, 406107 words/s, in_qsize 8, out_qsize 0
2019-11-21 18:08:46,252 : INFO : EPOCH 17 - PROGRESS: at 19.89% examples, 408687 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:08:47,263 : INFO : EPOCH 17 - PROGRESS: at 21.71% examples, 409625 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:08:48,266 : INFO : EPOCH 17 - PROGRESS: at 23.57% examples, 410685 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:08:49,268 : INFO : EPOCH 17 - PROGRESS: at 25.07% examples, 406326 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:08:50,299 : INFO : EPOCH 17 - PROGRESS: at 26.77% examples, 404257 words/s, in_qsize 8, out_qsize 0
2019-11-21 18:08:51,308 : INFO : EPOCH 17 - PROGRESS: at 28.71% examples, 406040 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:08:52,325 : INFO : EPOCH 17 - PROGRESS: at 30.59% examples, 406879 words/s

2019-11-21 18:09:54,197 : INFO : EPOCH 18 - PROGRESS: at 42.36% examples, 417176 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:09:55,197 : INFO : EPOCH 18 - PROGRESS: at 44.22% examples, 417514 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:09:56,263 : INFO : EPOCH 18 - PROGRESS: at 46.09% examples, 417571 words/s, in_qsize 8, out_qsize 1
2019-11-21 18:09:57,270 : INFO : EPOCH 18 - PROGRESS: at 48.03% examples, 418138 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:09:58,273 : INFO : EPOCH 18 - PROGRESS: at 49.91% examples, 418722 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:09:59,280 : INFO : EPOCH 18 - PROGRESS: at 51.77% examples, 418931 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:10:00,284 : INFO : EPOCH 18 - PROGRESS: at 53.54% examples, 418421 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:10:01,312 : INFO : EPOCH 18 - PROGRESS: at 55.43% examples, 418838 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:10:02,334 : INFO : EPOCH 18 - PROGRESS: at 57.39% examples, 419271 words/s

2019-11-21 18:11:04,365 : INFO : EPOCH 19 - PROGRESS: at 70.21% examples, 416538 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:05,385 : INFO : EPOCH 19 - PROGRESS: at 72.12% examples, 416827 words/s, in_qsize 8, out_qsize 1
2019-11-21 18:11:06,400 : INFO : EPOCH 19 - PROGRESS: at 73.99% examples, 416954 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:07,405 : INFO : EPOCH 19 - PROGRESS: at 75.93% examples, 417171 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:08,416 : INFO : EPOCH 19 - PROGRESS: at 77.83% examples, 417509 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:09,439 : INFO : EPOCH 19 - PROGRESS: at 79.58% examples, 417082 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:10,443 : INFO : EPOCH 19 - PROGRESS: at 81.38% examples, 416833 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:11,514 : INFO : EPOCH 19 - PROGRESS: at 83.14% examples, 415713 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:11:12,552 : INFO : EPOCH 19 - PROGRESS: at 84.82% examples, 414945 words/s

2019-11-21 18:12:13,723 : INFO : EPOCH 20 - PROGRESS: at 96.03% examples, 417189 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:12:14,739 : INFO : EPOCH 20 - PROGRESS: at 97.89% examples, 417421 words/s, in_qsize 7, out_qsize 0
2019-11-21 18:12:15,760 : INFO : EPOCH 20 - PROGRESS: at 99.83% examples, 417593 words/s, in_qsize 4, out_qsize 0
2019-11-21 18:12:15,773 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-21 18:12:15,812 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-21 18:12:15,823 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-21 18:12:15,832 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-21 18:12:15,834 : INFO : EPOCH - 20 : training on 23279529 raw words (22951015 effective words) took 54.9s, 417720 effective words/s
2019-11-21 18:12:15,835 : INFO : training on a 465590580 raw words (459020300 effective words) took 2227.7s, 206048 effective words/s



Evaluating Doc2Vec(dm/c,d100,n5,w5,mc2,t4)

0.306080 Doc2Vec(dm/c,d100,n5,w5,mc2,t4)


Evaluating Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4)

0.104360 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4)


Evaluating Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/c,d100,n5,w5,mc2,t4)

0.104840 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/c,d100,n5,w5,mc2,t4)



Achieved Sentiment-Prediction Accuracy
--------------------------------------
Compare error rates achieved, best-to-worst



In [20]:
print("Err_rate Model")
for rate, name in sorted((rate, name) for name, rate in error_rates.items()):
    print("%f %s" % (rate, name))

Err_rate Model
0.104360 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4)
0.104840 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/c,d100,n5,w5,mc2,t4)
0.105920 Doc2Vec(dbow,d100,n5,mc2,t4)
0.174240 Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4)
0.306080 Doc2Vec(dm/c,d100,n5,w5,mc2,t4)


In [ ]:
print("Accuracy")

for acc,name in sorted((acc,name) for name, acc in error)

In our testing, contrary to the results of the paper, on this problem,
PV-DBOW alone performs as good as anything else. Concatenating vectors from
different models only sometimes offers a tiny predictive improvement – and
stays generally close to the best-performing solo model included.

The best results achieved here are just around 10% error rate, still a long
way from the paper's reported 7.42% error rate.

(Other trials not shown, with larger vectors and other changes, also don't
come close to the paper's reported value. Others around the net have reported
a similar inability to reproduce the paper's best numbers. The PV-DM/C mode
improves a bit with many more training epochs – but doesn't reach parity with
PV-DBOW.)




Examining Results
-----------------

Let's look for answers to the following questions:

#. Are inferred vectors close to the precalculated ones?
#. Do close documents seem more related than distant ones?
#. Do the word vectors show useful similarities?
#. Are the word vectors from this dataset any good at analogies?




Are inferred vectors close to the precalculated ones?
-----------------------------------------------------



In [21]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # Pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

2019-11-21 18:15:28,126 : INFO : precomputing L2-norms of doc weight vectors


for doc 71229...


2019-11-21 18:15:28,518 : INFO : precomputing L2-norms of doc weight vectors
2019-11-21 18:15:28,648 : INFO : precomputing L2-norms of doc weight vectors


Doc2Vec(dbow,d100,n5,mc2,t4):
 [(71229, 0.9894205331802368), (49613, 0.5576101541519165), (20334, 0.5438925623893738)]
Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4):
 [(71229, 0.9080021977424622), (49613, 0.5746079087257385), (80528, 0.5528310537338257)]
Doc2Vec(dm/c,d100,n5,w5,mc2,t4):
 [(71229, 0.7498977184295654), (61396, 0.4301270842552185), (3886, 0.42097970843315125)]


(Yes, here the stored vector from 20 epochs of training is usually one of the
closest to a freshly-inferred vector for the same words. Defaults for
inference may benefit from tuning for each dataset or model parameters.)




Do close documents seem more related than distant ones?
-------------------------------------------------------



In [22]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    s = sims[index]
    i = sims[index][0]
    words = ' '.join(alldocs[i].words)
    print(u'%s %s: «%s»\n' % (label, s, words))

TARGET (20907): «Being a fan of time travel stories I was surprised that there was no 'device' that sent Russell Johnson's character through time. He just appeared in 1865. It was a disappointing part of the episode. I enjoyed the premise of the dangers of 'altering' the future by changing the past. Other Twilight Zone episodes about time travel such as "No Time Like The Past", "Once Upon A Time", etc. were more to my liking because of the uses of time travel 'devices'. Perhaps if Russell Johnson's character had been the same character he played in "Execution" it would have been more acceptable to fans like myself. As sci-fi fans know, no characters really ever have to 'die' in time travel episodes. All sorts of plot 'twists' can be applied in these types of stories. That was the only flaw I could comment on for this episode.»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/c,d100,n5,w5,mc2,t4):

MOST (90064, 0.4897709786891937): «Maybe it's because I just recently got into Indian soap o

Somewhat, in terms of reviewer tone, movie genre, etc... the MOST
cosine-similar docs usually seem more like the TARGET than the MEDIAN or
LEAST... especially if the MOST has a cosine-similarity > 0.5. Re-run the
cell to try another random target document.




Do the word vectors show useful similarities?
---------------------------------------------




In [23]:
import random

word_models = simple_models[:]

def pick_random_word(model, threshold=10):
    # pick a random word with a suitable number of occurences
    while True:
        word = random.choice(model.wv.index2word)
        if model.wv.vocab[word].count > threshold:
            return word

target_word = pick_random_word(word_models[0])
# or uncomment below line, to just pick a word from the relevant domain:
# target_word = 'comedy/drama'

for model in word_models:
    print('target_word: %r model: %s similar words:' % (target_word, model))
    for i, (word, sim) in enumerate(model.wv.most_similar(target_word, topn=10), 1):
        print('    %d. %.2f %r' % (i, sim, word))
    print()

2019-11-21 18:20:41,863 : INFO : precomputing L2-norms of word weight vectors


target_word: '(Ron' model: Doc2Vec(dbow,d100,n5,mc2,t4) similar words:


2019-11-21 18:20:42,168 : INFO : precomputing L2-norms of word weight vectors


    1. 0.47 'attests'
    2. 0.45 'Beach,"'
    3. 0.42 'parody?'
    4. 0.42 'unite'
    5. 0.41 'Jansen,'
    6. 0.41 '(wonderful!)'
    7. 0.41 'scamming'
    8. 0.40 "Talia's"
    9. 0.40 'Aeris'
    10. 0.40 'Russo),'

target_word: '(Ron' model: Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4) similar words:


2019-11-21 18:20:42,411 : INFO : precomputing L2-norms of word weight vectors


    1. 0.55 'Smith),'
    2. 0.54 'Ford),'
    3. 0.51 '(Gordon'
    4. 0.51 'Poe.<br'
    5. 0.50 '(Humphrey'
    6. 0.50 '(Willie'
    7. 0.50 'Boone,'
    8. 0.50 'Stein,'
    9. 0.49 'Osborne,'
    10. 0.49 'Curtis)'

target_word: '(Ron' model: Doc2Vec(dm/c,d100,n5,w5,mc2,t4) similar words:
    1. 0.67 '(Timothy'
    2. 0.66 '(Gary'
    3. 0.66 '(Ben'
    4. 0.63 '(Linda'
    5. 0.63 '(Morgan'
    6. 0.63 '(Edward'
    7. 0.63 '(Jack'
    8. 0.63 'Hobbes,'
    9. 0.63 '(Christopher'
    10. 0.62 '(Isabelle'



Do the DBOW words look meaningless? That's because the gensim DBOW model
doesn't train word vectors – they remain at their random initialized values –
unless you ask with the ``dbow_words=1`` initialization parameter. Concurrent
word-training slows DBOW mode significantly, and offers little improvement
(and sometimes a little worsening) of the error rate on this IMDB
sentiment-prediction task, but may be appropriate on other tasks, or if you
also need word-vectors.

Words from DM models tend to show meaningfully similar words when there are
many examples in the training data (as with 'plot' or 'actor'). (All DM modes
inherently involve word-vector training concurrent with doc-vector training.)




Are the word vectors from this dataset any good at analogies?
-------------------------------------------------------------



In [25]:
# grab the file if not already local
questions_filename = 'questions-words.txt'
if not os.path.isfile(questions_filename):
    # Download IMDB archive
    print("Downloading analogy questions file...")
    url = u'https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt'
    with smart_open.open(url, 'rb') as fin:
        with smart_open.open(questions_filename, 'wb') as fout:
            fout.write(fin.read())
assert os.path.isfile(questions_filename), "questions-words.txt unavailable"
print("Success, questions-words.txt is available for next steps.")

# Note: this analysis takes many minutes
for model in word_models:
    score, sections = model.wv.evaluate_word_analogies('questions-words.txt')
    correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
    print('%s: %0.2f%% correct (%d of %d)' % (model, float(correct*100)/(correct+incorrect), correct, correct+incorrect))

Success, questions-words.txt is available for next steps.


2019-11-21 18:21:09,211 : INFO : Evaluating word analogies for top 300000 words in the model on questions-words.txt
2019-11-21 18:21:15,694 : INFO : capital-common-countries: 0.0% (0/420)
2019-11-21 18:21:26,841 : INFO : capital-world: 0.0% (0/902)
2019-11-21 18:21:27,792 : INFO : currency: 0.0% (0/86)
2019-11-21 18:21:45,032 : INFO : city-in-state: 0.0% (0/1510)
2019-11-21 18:21:51,400 : INFO : family: 0.0% (0/506)
2019-11-21 18:22:02,292 : INFO : gram1-adjective-to-adverb: 0.0% (0/992)
2019-11-21 18:22:10,881 : INFO : gram2-opposite: 0.0% (0/756)
2019-11-21 18:22:25,265 : INFO : gram3-comparative: 0.0% (0/1332)
2019-11-21 18:22:36,619 : INFO : gram4-superlative: 0.0% (0/1056)
2019-11-21 18:22:47,368 : INFO : gram5-present-participle: 0.0% (0/992)
2019-11-21 18:23:02,669 : INFO : gram6-nationality-adjective: 0.0% (0/1445)
2019-11-21 18:23:19,227 : INFO : gram7-past-tense: 0.0% (0/1560)
2019-11-21 18:23:31,653 : INFO : gram8-plural: 0.0% (0/1190)
2019-11-21 18:23:41,259 : INFO : gram9-

Doc2Vec(dbow,d100,n5,mc2,t4): 0.00% correct (0 of 13617)


2019-11-21 18:23:42,263 : INFO : Evaluating word analogies for top 300000 words in the model on questions-words.txt
2019-11-21 18:23:46,695 : INFO : capital-common-countries: 3.3% (14/420)
2019-11-21 18:23:57,115 : INFO : capital-world: 0.4% (4/902)
2019-11-21 18:23:58,037 : INFO : currency: 0.0% (0/86)
2019-11-21 18:24:14,259 : INFO : city-in-state: 0.1% (2/1510)
2019-11-21 18:24:20,041 : INFO : family: 37.2% (188/506)
2019-11-21 18:24:31,152 : INFO : gram1-adjective-to-adverb: 2.8% (28/992)
2019-11-21 18:24:39,537 : INFO : gram2-opposite: 5.6% (42/756)
2019-11-21 18:24:56,137 : INFO : gram3-comparative: 49.2% (656/1332)
2019-11-21 18:25:08,873 : INFO : gram4-superlative: 24.5% (259/1056)
2019-11-21 18:25:21,103 : INFO : gram5-present-participle: 23.3% (231/992)
2019-11-21 18:25:38,774 : INFO : gram6-nationality-adjective: 2.4% (35/1445)
2019-11-21 18:25:59,006 : INFO : gram7-past-tense: 28.1% (438/1560)
2019-11-21 18:26:12,183 : INFO : gram8-plural: 16.9% (201/1190)
2019-11-21 18:26:

Doc2Vec("alpha=0.05",dm/m,d100,n5,w10,mc2,t4): 18.15% correct (2472 of 13617)


2019-11-21 18:26:24,098 : INFO : Evaluating word analogies for top 300000 words in the model on questions-words.txt
2019-11-21 18:26:29,037 : INFO : capital-common-countries: 2.4% (10/420)
2019-11-21 18:26:38,397 : INFO : capital-world: 0.3% (3/902)
2019-11-21 18:26:39,348 : INFO : currency: 0.0% (0/86)
2019-11-21 18:26:55,112 : INFO : city-in-state: 0.3% (5/1510)
2019-11-21 18:27:00,694 : INFO : family: 37.4% (189/506)
2019-11-21 18:27:11,203 : INFO : gram1-adjective-to-adverb: 7.4% (73/992)
2019-11-21 18:27:19,545 : INFO : gram2-opposite: 3.7% (28/756)
2019-11-21 18:27:34,592 : INFO : gram3-comparative: 36.5% (486/1332)
2019-11-21 18:27:46,404 : INFO : gram4-superlative: 28.0% (296/1056)
2019-11-21 18:27:57,470 : INFO : gram5-present-participle: 34.4% (341/992)
2019-11-21 18:28:14,761 : INFO : gram6-nationality-adjective: 2.0% (29/1445)
2019-11-21 18:28:32,915 : INFO : gram7-past-tense: 26.0% (406/1560)
2019-11-21 18:28:47,032 : INFO : gram8-plural: 9.4% (112/1190)
2019-11-21 18:28:5

Doc2Vec(dm/c,d100,n5,w5,mc2,t4): 17.69% correct (2409 of 13617)


Even though this is a tiny, domain-specific dataset, it shows some meager
capability on the general word analogies – at least for the DM/mean and
DM/concat models which actually train word vectors. (The untrained
random-initialized words of the DBOW model of course fail miserably.)


